In [16]:
import pandas as pd
import sys
sys.path.append("../")
from importlib import reload
import model
reload(model)
from model import *
from date import *

In [40]:
end = get_date_index_parse('2017-08-15')
num = 8
ranges = []
for i in range(num):
    week2end = end
    week2start = end-13
    week1end = end-14
    week1start = end-35
    ranges.append((list(range(week2start, week2end+1)), list(range(week1start, week1end+1))))
    end -= 14
week1start

1554

In [27]:
get_index_date(week1start)

datetime.datetime(2017, 4, 4, 0, 0)

In [28]:
df = load_data_in_date_range('./../data/train_processed.csv', '2017-04-04', '2017-08-15', 110000000)

Reading batch from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 8659998, mapping...
Mapped, reducing...
Batch done.
Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [29]:
df = convert_unit_sales(df)
df = fill_empty_sales(df)
df.head()

In [49]:
def fill_two_weeks(df, df_prev):
    mean_agg = df_prev.groupby(['item_nbr', 'store_nbr', 'onpromotion'], as_index=False).agg({'unit_sales':'mean'})
    mean_agg.rename(columns={'unit_sales':'unit_sales_mean'}, inplace=True)
    df = df.merge(mean_agg, on=['item_nbr','store_nbr', 'onpromotion'], how='left')
    
    df_prev.date = df_prev.date + 14
    df_prev['unit_sales_prev14'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev14']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date + 1
    df_prev['unit_sales_prev15'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev15']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date + 1
    df_prev['unit_sales_prev16'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev16']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date + 1
    df_prev['unit_sales_prev17'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev17']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date + 1
    df_prev['unit_sales_prev18'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev18']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date - 5
    df_prev['unit_sales_prev13'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev13']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df_prev.date = df_prev.date - 1
    df_prev['unit_sales_prev12'] = df_prev['unit_sales']
    df = df.merge(df_prev[['item_nbr','store_nbr', 'date', 'unit_sales_prev12']], on=['item_nbr','store_nbr', 'date'], how='left')
    
    df.fillna(0.0, inplace=True)
    return df

result = []
for week2, week1 in reversed(ranges):
    week2df = df[df.date.isin(week2)]
    week1df = df[df.date.isin(week1)]
    week2df = fill_two_weeks(week2df, week1df)
    result.append(week2df)

C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [50]:
result[0].head(20)

,date,store_nbr,item_nbr,id,unit_sales,onpromotion,unit_sales_mean,unit_sales_prev14,unit_sales_prev15,unit_sales_prev16,unit_sales_prev17,unit_sales_prev18,unit_sales_prev13,unit_sales_prev12
0,1576,46,1695836,113723318.0,3.983762,True,3.759474,3.736180,2.562543,2.337137,4.329911,0.000000,2.519459,0.000000
1,1576,46,1695837,113723319.0,2.596886,True,1.755228,1.583318,1.061135,2.051215,2.390052,2.177057,1.610999,1.568372
2,1576,46,1695840,113723320.0,1.098612,True,1.730986,1.791759,1.791759,1.945910,1.609438,1.945910,2.708050,2.197225
3,1576,46,1695845,113723321.0,3.044523,True,3.595048,3.713572,3.135494,3.178054,3.970292,3.688879,3.784190,3.610918
4,1576,46,1695846,113723322.0,1.583318,True,1.075955,1.362078,1.232469,0.733813,1.667117,2.067159,1.029898,1.059444
5,1576,46,1695872,0.0,0.000000,False,1.210303,1.386294,0.693147,2.197225,2.564949,3.135494,2.639057,1.098612
6,1576,46,1695873,113723324.0,1.945910,True,1.805367,1.609438,1.945910,1.609438,1.098612,2.484907,0.000000,0.000000
7,1576,46,1695874,0.0,0.000000,False,1.106828,1.791759,1.791759,2.564949,2.890372,2.484907,0.000000,0.000000
8,1576,46,1695875,113723325.0,2.302585,True,2.138333,1.791759,2.302585,1.609438,2.079442,2.995732,0.000000,0.000000
9,1576,46,1695882,113723326.0,1.811741,True,2.227654,1.848086,1.734141,1.561838,2.417587,1.973430,1.190603,1.970712


In [43]:
new_df = pd.concat(result)

In [44]:
new_df['date'].min()

1576

In [45]:
new_df['date'].max()

1687

In [46]:
get_index_date(new_df['date'].max())

datetime.datetime(2017, 8, 15, 0, 0)